# Build a Spam Classifier with Keras
With deep learning and AI, handling spam content has gotten easier and easier. Over time (and with the aid of direct user feedback) our spam classifier will rarely produce erroneous results. 

This is the first part of a multi-part series covering how to:

- Build an AI Model (this one)
- Integrate a NoSQL Database (inference result storing)
- Deploy an AI Model into Production

### Prerequisites
- Prepare your dataset using [this notebook](https://github.com/codingforentrepreneurs/AI-as-an-API/blob/main/guides/spam-classifier/1%20-%20Prepare%20the%20AI%20Spam%20Classifier%20Dataset.ipynb) .
- Convert your dataset into trainable vectors in [this notebook](https://github.com/codingforentrepreneurs/AI-as-an-API/blob/main/guides/spam-classifier/2%20-%20Convert%20Dataset%20into%20Vectors.ipynb) (Either way, this notebook will run this step for us).


### Running this notebook:
- Recommended: Use [Colab](https://colab.research.google.com/github/codingforentrepreneurs/AI-as-an-API/blob/main/guides/spam-classifier/Spam_Classifier_with_Keras.ipynb) as it offers free GPUs for training models. [Launch this notebook here]([Colab](https://colab.research.google.com/github/codingforentrepreneurs/AI-as-an-API/blob/main/guides/spam-classifier/Spam_Classifier_with_Keras.ipynb))
- Fork [the AI as an API repo](https://github.com/codingforentrepreneurs/AI-as-an-API) and run `guides/spam-classifier/Spam_Classifier_with_Keras.ipynb` whenever you'd like.

This notebook is brought to in you in partnership with [DataStax](https://dtsx.io/3nRWZEG).

In [1]:
!c:\Users\Acer\Documents\Visual-Studio\MACHINE-LEARNING\ai-api\Scripts\python.exe -m pip install boto3
# !pip install pandas tensorflow

The system cannot find the path specified.


In [2]:
import boto3
import os
import pathlib
import pandas as pd
import pickle

In [3]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

data = {}

with open(TRAINING_DATA_PATH, 'rb') as f:
    data = pickle.load(f)


In [4]:
BASE_DIR = pathlib.Path(os.path.abspath('')).resolve().parent.parent

EXPORT_DIR = BASE_DIR / "datasets/exports/"
GUIDES_DIR = BASE_DIR / "/guides/spam-classifier/"
DATASET_CSV_PATH = EXPORT_DIR / 'spam-dataset.csv'
TRAINING_DATA_PATH = EXPORT_DIR / 'spam-training-data.pkl'
PART_TWO_GUIDE_PATH = GUIDES_DIR / '2\ -\ Convert\ Dataset\ into\ Vectors.ipynb'

E:\guides\spam-classifier\2\ -\ Convert\ Dataset\ into\ Vectors.ipynb


## Prepare Dataset

Creating a dataset rarely happens next to where you run the training. The below cells are a method for us to extract the needed data to perform training against.

In [5]:
!mkdir -p "$EXPORT_DIR"
!mkdir -p "$GUIDES_DIR"
!curl "https://raw.githubusercontent.com/codingforentrepreneurs/AI-as-an-API/main/datasets/exports/spam-dataset.csv" -o "$DATASET_CSV_PATH"
!curl "https://raw.githubusercontent.com/codingforentrepreneurs/AI-as-an-API/main/guides/spam-classifier/2%20-%20Convert%20Dataset%20into%20Vectors.ipynb" -o "$PART_TWO_GUIDE_PATH"

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file E:\code\SpamDetector_FastAPI\AI-as-an-API\datasets\exports already exists.
Error occurred while processing: E:\code\SpamDetector_FastAPI\AI-as-an-API\datasets\exports.
A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file E:\guides\spam-classifier already exists.
Error occurred while processing: E:\guides\spam-classifier.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 64  729k   64  472k    0     0   616k      0  0:00:01 --:--:--  0:00:01  616k
100  729k  100  729k    0     0   861k      0 --:--:-- --:--:-- --:--:--  861k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   T

Let's review our extracted dataset which combines two different spam datasets as outlined in [this notebook](https://github.com/codingforentrepreneurs/AI-as-an-API/blob/main/guides/spam-classifier/1%20-%20Prepare%20the%20AI%20Spam%20Classifier%20Dataset.ipynb).

In [6]:
df = pd.read_csv(DATASET_CSV_PATH)
df.head()

,label,text,source
0,ham,"Go until jurong point, crazy.. Available only ...",uci-spam-sms
1,ham,Ok lar... Joking wif u oni...,uci-spam-sms
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,uci-spam-sms
3,ham,U dun say so early hor... U c already then say...,uci-spam-sms
4,ham,"Nah I don't think he goes to usf, he lives aro...",uci-spam-sms


Extract prepared training dataset results.

In [7]:
data = {}

with open(TRAINING_DATA_PATH, 'rb') as f:
    data = pickle.load(f)

> While the above code uses `pickle` to load in data, this data is actually exported via `pickle` when we execute the `%run` only a few steps ago. Since `pickle` can be unsafe to use from third-party downloaded data, we actually generate (again using `%run`) this pickle data and therefore is safe to use -- it's never downloaded.

## Transform Extracted Dataset

In [8]:
X_test = data['X_test']
X_train = data['X_train']
y_test = data['y_test']
y_train = data['y_train']
labels_legend_inverted = data['labels_legend_inverted']
legend = data['legend']
max_sequence = data['max_sequence']
max_words = data['max_words']
tokenizer = data['tokenizer']

## Create our LSTM Model

In [9]:
embed_dim = 128
lstm_out = 196
MAX_NUM_WORDS=280

model = Sequential()
model.add(Embedding(MAX_NUM_WORDS, embed_dim, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

batch_size = 32
epochs = 5
model.fit(X_train, y_train, validation_data=(X_test, y_test),
          batch_size=batch_size, verbose=1, epochs=epochs)

MODEL_EXPORT_PATH = EXPORT_DIR / 'spam-model.h5'
model.save(str(MODEL_EXPORT_PATH))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 280, 128)          35840     
                                                                 
 spatial_dropout1d (SpatialD  (None, 280, 128)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 291,034
Trainable params: 291,034
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
batch_size = 32
epochs = 5
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, verbose=1, epochs=epochs)

Epoch 1/5
154/154 [==============================] - 220s 1s/step - loss: 0.2731 - accuracy: 0.8924 - val_loss: 0.1422 - val_accuracy: 0.9553
Epoch 2/5
154/154 [==============================] - 229s 1s/step - loss: 0.1115 - accuracy: 0.9596 - val_loss: 0.1058 - val_accuracy: 0.9661
Epoch 3/5
154/154 [==============================] - 231s 1s/step - loss: 0.0891 - accuracy: 0.9698 - val_loss: 0.0918 - val_accuracy: 0.9719
Epoch 4/5
154/154 [==============================] - 232s 2s/step - loss: 0.0815 - accuracy: 0.9741 - val_loss: 0.0950 - val_accuracy: 0.9698
Epoch 5/5
154/154 [==============================] - 230s 1s/step - loss: 0.0715 - accuracy: 0.9772 - val_loss: 0.0977 - val_accuracy: 0.9698


In [11]:
MODEL_EXPORT_PATH = EXPORT_DIR / 'spam-model.h5'
model.save(str(MODEL_EXPORT_PATH))

## Predict new data

In [12]:
import numpy as np

def predict(text_str, max_words=280, max_sequence = 280, tokenizer=None):
  if not tokenizer:
    return None
  sequences = tokenizer.texts_to_sequences([text_str])
  x_input = pad_sequences(sequences, maxlen=max_sequence)
  y_output = model.predict(x_input)
  top_y_index = np.argmax(y_output)
  preds = y_output[top_y_index]
  labeled_preds = [{f"{labels_legend_inverted[str(i)]}": x} for i, x in enumerate(preds)]
  return labeled_preds

In [13]:
predict("Hello world", max_words=max_words, max_sequence=max_sequence, tokenizer=tokenizer)

1/1 [==============================] - 1s 725ms/step


[{'ham': 0.9938113}, {'spam': 0.006188751}]

## Exporting Tokenizer & Metadata

In [14]:
import json
metadata = {
    "labels_legend_inverted": labels_legend_inverted,
    "legend": legend,
    "max_sequence": max_sequence,
    "max_words": max_words,
}

METADATA_EXPORT_PATH = EXPORT_DIR / 'spam-classifer-metadata.json'
METADATA_EXPORT_PATH.write_text(json.dumps(metadata, indent=4))

187

In [15]:
tokenizer_as_json = tokenizer.to_json()

TOKENIZER_EXPORT_PATH = EXPORT_DIR / 'spam-classifer-tokenizer.json'
TOKENIZER_EXPORT_PATH.write_text(tokenizer_as_json)

828992